In [25]:
import cv2
import numpy as np

def NMS(lists, thre):
    """
    Non-max Suppression Algorithm
    @param  Lists  lists[0:4]: x1, y1, x2, y2;
    @param  thre   float IoU threshold
    @return Selected boxes after nms operation a
    @return Score of the selected boxes
    """
# lists is a list.  lists[4]: score
    if len(lists) == 0:
        return [], []
    
    # Bounding boxes and scores
    # Important: convert the 2d list into 2d np array.
    box = np.array([item[:4] for item in lists])
    probability = np.array([item[4] for item in lists])
    
    # Extract the coordinates
    start_x = box[:,0]
    end_x   = box[:,2]
    start_y = box[:,1]
    end_y   = box[:,3]
    
    # Extract the score
    score = np.array(probability)
    
    # list to store the selected bounding boxes
    picked_box = []
    picked_score = []
    
    # Area of the bounding boxes
    areas = (end_x - start_x) * (end_y - start_y)
    
    # ascending order of the index based on the score
    order = np.argsort(score)
    
    while len(order) > 0:
        # The most confident box at each stage
        index = order[-1]
        
        # Store it to the final selected list
        picked_box.append(box[index])
        picked_score.append(score[index])
        
        # Eliminate the Non-maximum boxes whose IOU is greater than the threshold
        # Vectorized implementation
        x1 = np.maximum(start_x[index], start_x[order[:-1]])
        x2 = np.minimum(end_x[index], end_x[order[:-1]])
        y1 = np.maximum(start_y[index], start_y[order[:-1]])
        y2 = np.minimum(end_y[index], end_y[order[:-1]])

        w = np.maximum(0.0, x2 - x1)
        h = np.maximum(0.0, y2 - y1)
        intersection = w * h
        
        # Compute the ratio between intersection and union
        ratio = intersection / (areas[index] + areas[order[:-1]] - intersection)
        
        remains = np.where(ratio < thre)
        order = order[remains]
        
    return picked_box, picked_score
        
        
# Image name
image_name = 'nms.jpg'

# Bounding boxes
bounding_boxes = [(187, 82, 337, 317), (150, 67, 305, 282), (246, 121, 368, 304)]
confidence_score = [0.9, 0.75, 0.8]

lists = [[187, 82, 337, 317, 0.9], [150, 67, 305, 282,  0.75], [246, 121, 368, 304, 0.8]]

# Read image
image = cv2.imread(image_name)

# Copy image as original
org = image.copy()

# Draw parameters
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
thickness = 2

# IoU threshold
threshold = 0.4

# Draw bounding boxes and confidence score
for (start_x, start_y, end_x, end_y), confidence in zip(bounding_boxes, confidence_score):
    (w, h), baseline = cv2.getTextSize(str(confidence), font, font_scale, thickness)
    # Fill in the color of the text
    cv2.rectangle(org, (start_x, start_y - (2 * baseline + 5)), (start_x + w, start_y), (0, 255, 255), -1)
    cv2.rectangle(org, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)
    cv2.putText(org, str(confidence), (start_x, start_y), font, font_scale, (0, 0, 0), thickness)

# Run non-max suppression algorithm
picked_boxes, picked_score = NMS(lists, threshold)

# Draw bounding boxes and confidence score after non-maximum supression
for (start_x, start_y, end_x, end_y), confidence in zip(picked_boxes, picked_score):
    (w, h), baseline = cv2.getTextSize(str(confidence), font, font_scale, thickness)
    cv2.rectangle(image, (start_x, start_y - (2 * baseline + 5)), (start_x + w, start_y), (0, 255, 255), -1)
    cv2.rectangle(image, (start_x, start_y), (end_x, end_y), (0, 255, 255), 2)
    cv2.putText(image, str(confidence), (start_x, start_y), font, font_scale, (0, 0, 0), thickness)

# Show image
cv2.imshow('Original', org)
cv2.imshow('NMS', image)
cv2.waitKey(5000)